# Balance dataset

In [ ]:
!pip install keras-tuner

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.utils import resample
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier  # <-- Added import for gradient boosting
import re
import numpy as np

# If you have not already installed keras_tuner:
# pip install keras-tuner --upgrade
import keras_tuner as kt

# -----------------------------
# 1. LOAD AND PREPROCESS DATA
# -----------------------------
train_df = pd.read_csv('train.csv').sample(n=1000, random_state=42)
test_df = pd.read_csv('test.csv')

def bucket_age(age_str):
    """Convert age strings (e.g., '2 years', '4 weeks') into age buckets."""
    if pd.isna(age_str):
        return "Unknown"

    age_str = age_str.lower().strip()
    conversion = {"year": 365, "month": 30, "week": 7, "day": 1}

    match = re.match(r"(\d+)\s*(year|month|week|day)s?", age_str)
    if match:
        num = int(match.group(1))
        unit = match.group(2)
        days = num * conversion[unit]

        if days <= 180:
            return "Baby"
        elif 181 <= days <= 730:
            return "Child"
        elif 1096 <= days < 4015:
            return "Adult"
        elif days >= 4015:
            return "Senior"

    return "Unknown"

# Apply age bucketing to the training set
train_df["Age Bucket"] = train_df["Age upon Intake"].apply(bucket_age)

# Clean Breed column: remove "mix" and extra whitespace
train_df['Breed_Clean'] = (
    train_df['Breed']
    .str.lower()
    .str.replace('mix', '', regex=False)
    .str.strip()
)

# Process Color column: convert to lowercase, remove extra spaces, etc.
train_df['Color_Clean'] = train_df['Color'].str.lower().str.strip()
color_split = train_df['Color_Clean'].str.split('/', expand=True)
train_df['Primary_Color'] = color_split[0]
train_df['Secondary_Color'] = (
    color_split[1] if color_split.shape[1] > 1 else np.nan
)

# Drop unnecessary columns
columns_to_drop = [
    'Name', 'Id', 'Intake Time', 'Outcome Time', 'Age upon Intake',
    'Date of Birth', 'Color', 'Found Location', 'Breed'
]
train_df.drop(
    columns=[col for col in columns_to_drop if col in train_df.columns],
    inplace=True
)

# Group rare categories into "Other"
min_count = 50  # Adjust this threshold as you see fit
rare_conditions = train_df['Intake Condition'].value_counts()
rare_conditions = rare_conditions[rare_conditions < min_count].index
train_df['Intake Condition'] = train_df['Intake Condition'].replace(rare_conditions, 'Other')

# -----------------------------
# 2. BALANCE THE TRAINING DATA (Oversampling)
# -----------------------------
adopted = train_df[train_df["Outcome Type"] == "Adoption"]
transferred = train_df[train_df["Outcome Type"] == "Transfer"]
euthanasia = train_df[train_df["Outcome Type"] == "Euthanasia"]
return_to_owner = train_df[train_df["Outcome Type"] == "Return to Owner"]
died = train_df[train_df["Outcome Type"] == "Died"]

# Find the largest class count
max_count = max(
    len(adopted),
    len(transferred),
    len(euthanasia),
    len(return_to_owner),
    len(died)
)

# Randomly oversample each class to match the largest class
adopted_oversampled = resample(
    adopted, replace=True, n_samples=max_count, random_state=42
)
transferred_oversampled = resample(
    transferred, replace=True, n_samples=max_count, random_state=42
)
euthanasia_oversampled = resample(
    euthanasia, replace=True, n_samples=max_count, random_state=42
)
return_to_owner_oversampled = resample(
    return_to_owner, replace=True, n_samples=max_count, random_state=42
)
died_oversampled = resample(
    died, replace=True, n_samples=max_count, random_state=42
)

balanced_train_df = pd.concat([
    adopted_oversampled,
    transferred_oversampled,
    euthanasia_oversampled,
    return_to_owner_oversampled,
    died_oversampled
])

# -----------------------------
# 3. ENCODE CATEGORICAL LABELS
# -----------------------------
label_encoder = LabelEncoder()
balanced_train_df["Outcome Type"] = label_encoder.fit_transform(
    balanced_train_df["Outcome Type"]
)

# Convert categorical variables to dummy variables
balanced_train_df = pd.get_dummies(balanced_train_df)

# Prepare features (X) and labels (y)
X = balanced_train_df.drop(columns=["Outcome Type"])
y = balanced_train_df["Outcome Type"]

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# -----------------------------
# 4. TRAIN/VALIDATION SPLIT
# -----------------------------
X_train, X_val, y_train, y_val = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

# ----------------------------------------------------
# 5. HYPERPARAMETER TUNING FOR THE NEURAL NETWORK (KerasTuner)
# ----------------------------------------------------
def build_model(hp):
    """
    Build a Keras model using hyperparameters from KerasTuner.
    Adjust the search space as desired.
    """
    model = keras.Sequential()

    # Choose the number of hidden layers
    num_hidden_layers = hp.Int('num_hidden_layers', min_value=1, max_value=3, step=1)

    # First layer (input_shape must match the number of features)
    model.add(
        layers.Dense(
            units=hp.Int('units_0', min_value=32, max_value=128, step=32),
            activation='relu',
            input_shape=(X_train.shape[1],)
        )
    )

    # Additional hidden layers
    for i in range(1, num_hidden_layers + 1):
        model.add(
            layers.Dense(
                units=hp.Int(f'units_{i}', min_value=32, max_value=128, step=32),
                activation='relu'
            )
        )

    # Output layer
    model.add(
        layers.Dense(
            len(label_encoder.classes_),
            activation='softmax'
        )
    )

    # Choose an optimizer
    hp_learning_rate = hp.Choice('learning_rate', [1e-3, 5e-4, 1e-4])
    optimizer = keras.optimizers.Adam(learning_rate=hp_learning_rate)

    model.compile(
        optimizer=optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# Instantiate a tuner. We'll use RandomSearch for demonstration.
tuner = kt.RandomSearch(
    hypermodel=build_model,
    objective='val_accuracy',
    max_trials=5,  # Increase or decrease based on how exhaustive you want your search
    executions_per_trial=1,
    overwrite=True,
    directory='my_dir',
    project_name='nn_tuning_demo'
)

# Perform hyperparameter search
tuner.search(
    X_train, y_train,
    epochs=10,  # fewer epochs for faster tuning
    validation_split=0.2,
    batch_size=32
)

# Retrieve the best model found by the tuner
best_hps_model = tuner.get_best_models(num_models=1)[0]

# Optional: re-train the best model with more epochs
history = best_hps_model.fit(
    X_train, y_train,
    epochs=20,  # now train longer
    batch_size=32,
    validation_data=(X_val, y_val)
)

train_accuracy_nn = history.history['accuracy'][-1]
val_accuracy_nn = history.history['val_accuracy'][-1]
print(f"Neural Network - Final Training Accuracy: {train_accuracy_nn:.4f}")
print(f"Neural Network - Final Validation Accuracy: {val_accuracy_nn:.4f}")

# ----------------------------------------------------
# 6. HYPERPARAMETER TUNING FOR RANDOM FOREST (GridSearchCV)
# ----------------------------------------------------
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5]
}

rf_base = RandomForestClassifier(random_state=42)

grid_search = GridSearchCV(
    estimator=rf_base,
    param_grid=param_grid,
    cv=3,              # 3-fold cross-validation
    scoring='accuracy',
    n_jobs=-1,         # Use all available CPU cores
    verbose=1
)

grid_search.fit(X_train, y_train)

best_rf_model = grid_search.best_estimator_

train_accuracy_rf = best_rf_model.score(X_train, y_train)
val_accuracy_rf = best_rf_model.score(X_val, y_val)
print(f"Random Forest - Best Params: {grid_search.best_params_}")
print(f"Random Forest - Final Training Accuracy: {train_accuracy_rf:.4f}")
print(f"Random Forest - Final Validation Accuracy: {val_accuracy_rf:.4f}")

# ----------------------------------------------------
# 6.5 HYPERPARAMETER TUNING FOR GRADIENT BOOSTING (GridSearchCV)
# ----------------------------------------------------
param_grid_gb = {
    'n_estimators': [100, 200],
    'learning_rate': [0.1, 0.05],
    'max_depth': [3, 5],
    'subsample': [1.0, 0.8]
}

gb_base = GradientBoostingClassifier(random_state=42)

grid_search_gb = GridSearchCV(
    estimator=gb_base,
    param_grid=param_grid_gb,
    cv=3,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

grid_search_gb.fit(X_train, y_train)

best_gb_model = grid_search_gb.best_estimator_

train_accuracy_gb = best_gb_model.score(X_train, y_train)
val_accuracy_gb = best_gb_model.score(X_val, y_val)
print(f"Gradient Boosting - Best Params: {grid_search_gb.best_params_}")
print(f"Gradient Boosting - Final Training Accuracy: {train_accuracy_gb:.4f}")
print(f"Gradient Boosting - Final Validation Accuracy: {val_accuracy_gb:.4f}")

# -----------------------------
# 7. PREPROCESS THE TEST DATA
# -----------------------------
test_df["Age Bucket"] = test_df["Age upon Intake"].apply(bucket_age)

test_df['Breed_Clean'] = (
    test_df['Breed']
    .str.lower()
    .str.replace('mix', '', regex=False)
    .str.strip()
)

# Same drop columns logic
test_df.drop(
    columns=[col for col in columns_to_drop if col in test_df.columns],
    inplace=True
)

# Replace rare intake conditions with "Other" if they appeared in the training set
test_df['Intake Condition'] = test_df['Intake Condition'].replace(rare_conditions, 'Other')

# Convert categorical variables to numerical
test_df = pd.get_dummies(test_df)

# Align columns with training data
test_df = test_df.reindex(columns=X.columns, fill_value=0)

# Scale the test data
test_df_scaled = StandardScaler().fit(scaler.mean_.reshape(1, -1)).transform(test_df)
# or simply: test_df_scaled = scaler.transform(test_df)

# ----------------------------------------------------
# 8. MAKE PREDICTIONS & SOFT VOTING
# ----------------------------------------------------

# -- Neural Network predictions (best tuned model)
pred_probs_nn = best_hps_model.predict(test_df_scaled)  # Probability predictions

# -- Random Forest predictions (best tuned model)
pred_probs_rf = best_rf_model.predict_proba(test_df_scaled)

# -- Gradient Boosting predictions (best tuned model)
pred_probs_gb = best_gb_model.predict_proba(test_df_scaled)

# Combine predictions using soft voting (average probabilities from NN, RF, and GB)
avg_probs = (pred_probs_nn + pred_probs_rf + pred_probs_gb) / 3
final_pred_classes = np.argmax(avg_probs, axis=1)

# Convert numeric predictions back to original class labels
final_pred_classes = label_encoder.inverse_transform(final_pred_classes)

# Create a DataFrame for submission or further analysis
test_predictions = pd.DataFrame({
    "Id": range(1, len(final_pred_classes) + 1),
    "Outcome Type": final_pred_classes
})

test_predictions.to_csv("test_predictions.csv", index=False)
print("Predictions saved to test_predictions.csv")


Trial 5 Complete [00h 03m 08s]
val_accuracy: 0.6399697065353394

Best val_accuracy So Far: 0.6786761283874512
Total elapsed time: 00h 17m 58s


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Epoch 1/20
3093/3093 ━━━━━━━━━━━━━━━━━━━━ 28s 8ms/step - accuracy: 0.7004 - loss: 0.7243 - val_accuracy: 0.6851 - val_loss: 0.7688
Epoch 2/20
3093/3093 ━━━━━━━━━━━━━━━━━━━━ 26s 8ms/step - accuracy: 0.7093 - loss: 0.7033 - val_accuracy: 0.6881 - val_loss: 0.7580
Epoch 3/20
3093/3093 ━━━━━━━━━━━━━━━━━━━━ 26s 8ms/step - accuracy: 0.7157 - loss: 0.6881 - val_accuracy: 0.6889 - val_loss: 0.7544
Epoch 4/20
3093/3093 ━━━━━━━━━━━━━━━━━━━━ 25s 8ms/step - accuracy: 0.7175 - loss: 0.6819 - val_accuracy: 0.6947 - val_loss: 0.7457
Epoch 5/20
3093/3093 ━━━━━━━━━━━━━━━━━━━━ 25s 8ms/step - accuracy: 0.7185 - loss: 0.6764 - val_accuracy: 0.7001 - val_loss: 0.7360
Epoch 6/20
3093/3093 ━━━━━━━━━━━━━━━━━━━━ 40s 8ms/step - accuracy: 0.7233 - loss: 0.6622 - val_accuracy: 0.7024 - val_loss: 0.7309
Epoch 7/20
3093/3093 ━━━━━━━━━━━━━━━━━━━━ 30s 10ms/step - accuracy: 0.7294 - loss: 0.6512 - val_accuracy: 0.6996 - val_loss: 0.7431
Epoch 8/20
3093/3093 ━━━━━━━━━━━━━━━━━━━━ 41s 10ms/step - accuracy: 0.7286 - loss:

KeyboardInterrupt: 